In [13]:
import nest_asyncio

nest_asyncio.apply()

# Initializing RAG

This will load the model, vectorstore, and index. If running for the first time, it will download the model and set up vectorstore which will take some time.

In [1]:
from settings import init_rag

init_rag()

e:\Capelin\personalized-chatbot-experiment\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:logger:[SUCCESS] RAG system initialized successfully.


# Loading Users data from json

In [6]:
import json
from pathlib import Path

try:
    file_path = Path('data/users.json')
    with open(file_path, 'r', encoding='utf-8') as file:
        users = json.load(file)
except FileNotFoundError:
    raise FileNotFoundError("users.json file not found in the data directory")
except json.JSONDecodeError:
    raise ValueError("Invalid JSON format in users.json file")

# Processing the data

In [10]:
processed_users = []

for user in users:
    metadata = {
        "user_id": user.get("user_id"),
    }

    writing_samples = user.get("writing_samples")
    for sample in writing_samples:
        processed_users.append({"text": sample, "metadata": metadata})

    conversation_history = user.get("conversation_history")
    for conversation in conversation_history:
        text = ""
        for key, value in conversation.items():
            text += f"{key}: {value}\n"
        processed_users.append({"text": text, "metadata": metadata})

In [12]:
processed_users[:5]

[{'text': 'In my opinion, technological advancements should be approached with careful ethical considerations.',
  'metadata': {'user_id': 'user_1'}},
 {'text': 'Data-driven decision-making is the backbone of a successful business strategy.',
  'metadata': {'user_id': 'user_1'}},
 {'text': 'While automation enhances efficiency, it is crucial to address potential job displacement concerns.',
  'metadata': {'user_id': 'user_1'}},
 {'text': 'question: What do you think about AI?\nresponse: AI is a transformative technology, but its implications must be thoroughly studied.\n',
  'metadata': {'user_id': 'user_1'}},
 {'text': 'question: Do you prefer structured learning or hands-on experience?\nresponse: A combination of both is ideal, but structured learning provides a strong foundation.\n',
  'metadata': {'user_id': 'user_1'}}]

# Constructing Nodes

In [14]:
from llama_index.core.schema import TextNode

nodes = []

for user in processed_users:
    node = TextNode(text=user.get("text"), metadata=user.get("metadata"))
    nodes.append(node)

In [15]:
len(nodes)

15

In [17]:
nodes[:5]

[TextNode(id_='510ba1ac-b2ad-497f-ba6b-cea3982dc69b', embedding=None, metadata={'user_id': 'user_1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='In my opinion, technological advancements should be approached with careful ethical considerations.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='c9f358d3-b032-4920-8460-8ae09f46c35b', embedding=None, metadata={'user_id': 'user_1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Data-driven decision-making is the backbone of a successful business strategy.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='275561a5-799b-474f-bb4e-de317072

# Generate Embeddings for each Node

In [18]:
from llama_index.core.settings import Settings

Settings.embed_model

HuggingFaceEmbedding(model_name='BAAI/bge-large-en-v1.5', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000001754E5F1050>, num_workers=None, max_length=512, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

In [19]:
for node in nodes:
    node_embedding = Settings.embed_model.get_text_embedding(node.get_content(metadata_mode="all"))
    node.embedding = node_embedding

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.64it/s]


In [20]:
nodes[0]

TextNode(id_='510ba1ac-b2ad-497f-ba6b-cea3982dc69b', embedding=[0.023007385432720184, -0.020619800314307213, -0.011358833871781826, 0.029063748195767403, -0.007101496681571007, 0.0005087736062705517, 0.006458232179284096, -0.0009337735828012228, 0.04637742042541504, 0.028635822236537933, -0.002091059461236, -0.012070433236658573, 0.02023123763501644, 0.008843165822327137, -0.036259252578020096, -0.02834475412964821, -0.007509326096624136, -0.04327116161584854, -0.02396222949028015, 0.019224511459469795, 0.01669926382601261, 0.039726562798023224, -0.058825429528951645, -0.009362713433802128, -0.05365251377224922, 0.025259042158722878, -0.0003391002246644348, -0.008629685267806053, 0.06296512484550476, 0.06520964950323105, -0.018675168976187706, 0.02926488034427166, 0.00915459543466568, -0.04229195788502693, -0.02557825855910778, -0.010460435412824154, 0.03553266450762749, -0.015331195667386055, -0.019486673176288605, -0.04336060211062431, 0.0017959325341507792, 0.018177254125475883, 0.0

# Load Nodes into a Vectorstore

In [21]:
from vectorstore import get_vectorstore

vectorstore = get_vectorstore()
vectorstore.add(nodes)

['510ba1ac-b2ad-497f-ba6b-cea3982dc69b',
 'c9f358d3-b032-4920-8460-8ae09f46c35b',
 '275561a5-799b-474f-bb4e-de317072ac04',
 '778d3ad8-55a4-43be-b4c0-a5045dd24e6e',
 'f480e598-850a-44fe-aa44-fe56a0c5d6e2',
 '7b520730-08c3-44dd-84f5-9c0c70502a86',
 '2121af80-13f0-4b83-9c08-8ecbd4e2c223',
 'd46b77bb-b587-4b90-923b-ee35e911b611',
 'd5a935a1-82b9-4ecb-a533-0e4c8f02de94',
 '625c87bc-ad11-496f-af26-b86b26dc5057',
 '4b2e0bdf-d2ba-4d5c-8325-0ff182b9c59e',
 'cf1ac107-2ceb-4085-8bfc-a3671443af7b',
 'f5e96768-3665-4039-adf3-3b029539b834',
 'daba1093-3253-4371-ad23-9ff2d95e7f07',
 '04badf2c-cef7-479b-841d-a4e4b766b418']

# Retrieving Documents

In [31]:
user_id = "user_1"

In [26]:
from query import retrieve_documents

retrieved_documents = retrieve_documents("What is AI?", user_id, 5)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Generated queries:
Here are 4 search queries related to the input query "What is AI?":
What is Artificial Intelligence definition?
What are the types of Artificial Intelligence?
How does Artificial Intelligence work?
What are the applications of Artificial Intelligence technology?


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.50it/s]


In [28]:
len(retrieved_documents)

2

In [30]:
retrieved_documents[1].text

'question: Do you prefer structured learning or hands-on experience?\nresponse: A combination of both is ideal, but structured learning provides a strong foundation.\n'